Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

1. Logistic Regression (SGD)

In [5]:
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # L2 loss
  loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 50.784683
Minibatch accuracy: 7.8%
Validation accuracy: 9.6%
Minibatch loss at step 500: 0.855555
Minibatch accuracy: 80.5%
Validation accuracy: 81.7%
Minibatch loss at step 1000: 0.792322
Minibatch accuracy: 78.9%
Validation accuracy: 81.4%
Minibatch loss at step 1500: 0.629912
Minibatch accuracy: 86.7%
Validation accuracy: 81.5%
Minibatch loss at step 2000: 0.555495
Minibatch accuracy: 89.1%
Validation accuracy: 81.2%
Minibatch loss at step 2500: 0.564809
Minibatch accuracy: 85.9%
Validation accuracy: 81.5%
Minibatch loss at step 3000: 0.849585
Minibatch accuracy: 82.0%
Validation accuracy: 81.8%
Test accuracy: 88.3%


Comparing to the test accuracy in assignment2 (85.6%), the addition l2 loss makes improvement.

2. Neural Network Model

In [7]:
batch_size = 128
num_hidden_nodes = 1024
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  weights2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  layer1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  logits = tf.matmul(layer1_train, weights2) + biases2
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # L2 loss
  loss = tf.reduce_mean(loss + beta*tf.nn.l2_loss(weights1) + beta*tf.nn.l2_loss(weights2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_layer1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_layer1, weights2) + biases2)
  test_layer1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_prediction = tf.nn.softmax(tf.matmul(test_layer1, weights2) + biases2)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3512.017090
Minibatch accuracy: 10.9%
Validation accuracy: 33.9%
Minibatch loss at step 500: 21.245752
Minibatch accuracy: 86.7%
Validation accuracy: 84.8%
Minibatch loss at step 1000: 0.877935
Minibatch accuracy: 82.0%
Validation accuracy: 83.5%
Minibatch loss at step 1500: 0.631373
Minibatch accuracy: 86.7%
Validation accuracy: 84.1%
Minibatch loss at step 2000: 0.553557
Minibatch accuracy: 90.6%
Validation accuracy: 83.1%
Minibatch loss at step 2500: 0.612273
Minibatch accuracy: 87.5%
Validation accuracy: 83.7%
Minibatch loss at step 3000: 0.806678
Minibatch accuracy: 82.8%
Validation accuracy: 83.6%
Test accuracy: 90.0%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

Copy above neuro network model. Make a smaller size (500) train_dataset and train_label.

In [9]:
batch_size = 128
num_hidden_nodes = 1024
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  weights2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  layer1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  logits = tf.matmul(layer1_train, weights2) + biases2
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # L2 loss
  loss = tf.reduce_mean(loss + beta*tf.nn.l2_loss(weights1) + beta*tf.nn.l2_loss(weights2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_layer1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_layer1, weights2) + biases2)
  test_layer1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_prediction = tf.nn.softmax(tf.matmul(test_layer1, weights2) + biases2)

In [10]:
num_steps = 3001

# Update below to demonstrate overfitting
train_dataset2 = train_dataset[:500]
train_labels2 = train_labels[:500]

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels2.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset2[offset:(offset + batch_size), :]
    batch_labels = train_labels2[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3545.987305
Minibatch accuracy: 4.7%
Validation accuracy: 33.8%
Minibatch loss at step 500: 21.006817
Minibatch accuracy: 100.0%
Validation accuracy: 78.5%
Minibatch loss at step 1000: 0.482367
Minibatch accuracy: 99.2%
Validation accuracy: 78.9%
Minibatch loss at step 1500: 0.307614
Minibatch accuracy: 99.2%
Validation accuracy: 78.6%
Minibatch loss at step 2000: 0.296076
Minibatch accuracy: 100.0%
Validation accuracy: 78.3%
Minibatch loss at step 2500: 0.287510
Minibatch accuracy: 100.0%
Validation accuracy: 78.4%
Minibatch loss at step 3000: 0.275811
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Test accuracy: 85.3%


From the output the minibatch accuracy becomes 100% but the test accuracy is worse.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [12]:
batch_size = 128
num_hidden_nodes = 1024
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  weights2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  layer1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)

  # Add dropout
  keep_prob = 0.5
  layer1_train_drop = tf.nn.dropout(layer1_train, keep_prob)
    
    
  logits = tf.matmul(layer1_train_drop, weights2) + biases2
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # L2 loss
  loss = tf.reduce_mean(loss + beta*tf.nn.l2_loss(weights1) + beta*tf.nn.l2_loss(weights2))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_layer1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_layer1, weights2) + biases2)
  test_layer1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_prediction = tf.nn.softmax(tf.matmul(test_layer1, weights2) + biases2)

In [13]:
num_steps = 3001

# Update below to demonstrate overfitting
train_dataset2 = train_dataset[:500]
train_labels2 = train_labels[:500]

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels2.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset2[offset:(offset + batch_size), :]
    batch_labels = train_labels2[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3673.050293
Minibatch accuracy: 6.2%
Validation accuracy: 39.1%
Minibatch loss at step 500: 21.178850
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 1000: 0.515096
Minibatch accuracy: 100.0%
Validation accuracy: 78.8%
Minibatch loss at step 1500: 0.353041
Minibatch accuracy: 99.2%
Validation accuracy: 78.9%
Minibatch loss at step 2000: 0.327785
Minibatch accuracy: 99.2%
Validation accuracy: 78.5%
Minibatch loss at step 2500: 0.322942
Minibatch accuracy: 99.2%
Validation accuracy: 78.8%
Minibatch loss at step 3000: 0.308668
Minibatch accuracy: 100.0%
Validation accuracy: 78.5%
Test accuracy: 85.6%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


Base on the NN model, update: decay learning rate, L2 loss(updated beta value), dropout.
Tried to add more NN layer but the result is worse.

In [53]:
batch_size = 128
num_hidden_nodes_1 = 1024
num_hidden_nodes_2 = 256
beta = 0.00001

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes_1], 
                                             stddev=tf.sqrt(2.0/(image_size * image_size))))
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes_1]))
  weights2 = tf.Variable(tf.truncated_normal([num_hidden_nodes_1, num_hidden_nodes_2], 
                                             stddev=tf.sqrt(2.0/num_hidden_nodes_1)))
  biases2 = tf.Variable(tf.zeros([num_hidden_nodes_2])) 
  weights3 = tf.Variable(tf.truncated_normal([num_hidden_nodes_2, num_labels], 
                                             stddev=tf.sqrt(2.0/num_hidden_nodes_1)))
  biases3 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  keep_prob_1 = 0.5
  keep_prob_2 = 0.5
  
  train_layer_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  train_layer_drop_1 = tf.nn.dropout(train_layer_1, keep_prob_1)
  train_layer_2 = tf.nn.relu(tf.matmul(train_layer_1, weights2) + biases2)
  train_layer_drop_2 = tf.nn.dropout(train_layer_2, keep_prob_2)

  logits = tf.matmul(train_layer_drop_2, weights3) + biases3

  # L2 loss
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss = tf.reduce_mean(loss + beta*tf.nn.l2_loss(weights1) + beta*tf.nn.l2_loss(weights2) 
                        + beta*tf.nn.l2_loss(weights3))

  #loss = tf.reduce_mean(loss + beta*tf.nn.l2_loss(weights1) + beta*tf.nn.l2_loss(weights3))
  
  # Optimizer.
  global_step = tf.Variable(0, trainable=False)
  starter_learning_rate = 0.5
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           130, 0.8, staircase=True)

  # Passing global_step to minimize() will increment it at each step.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
    
  valid_layer_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_layer_2 = tf.nn.relu(tf.matmul(valid_layer_1, weights2) + biases2)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_layer_2, weights3) + biases3)
    
  test_layer_1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_layer_2 = tf.nn.relu(tf.matmul(test_layer_1, weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(test_layer_2, weights3) + biases3)

In [54]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("Learning rate: %.4f" % learning_rate.eval())
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.373571
Minibatch accuracy: 7.8%
Validation accuracy: 28.1%
Learning rate: 0.5000
Minibatch loss at step 500: 0.487679
Minibatch accuracy: 86.7%
Validation accuracy: 86.0%
Learning rate: 0.2560
Minibatch loss at step 1000: 0.435944
Minibatch accuracy: 84.4%
Validation accuracy: 87.1%
Learning rate: 0.1049
Minibatch loss at step 1500: 0.403189
Minibatch accuracy: 89.1%
Validation accuracy: 87.6%
Learning rate: 0.0429
Minibatch loss at step 2000: 0.294543
Minibatch accuracy: 91.4%
Validation accuracy: 87.8%
Learning rate: 0.0176
Minibatch loss at step 2500: 0.322248
Minibatch accuracy: 90.6%
Validation accuracy: 87.8%
Learning rate: 0.0072
Minibatch loss at step 3000: 0.445809
Minibatch accuracy: 85.9%
Validation accuracy: 87.8%
Learning rate: 0.0030
Test accuracy: 93.7%
